Data for sales/penetration will not be ready until after the 8th of the month.

### ***Pull Construction Funding at the beginning of the month, only once a month

December 2022 funding data - already pulled 11/30/22 - do not pull until after 12/31

In [3]:
import os
import sys
import pandas as pd
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

ProgrammingError: Oracle Client library has already been initialized

In [ ]:
# FRAM001P
conn2 = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

#CCDWO1P - SALES
conn3 = cx_Oracle.connect(
    user="AD21236",
    password="C#ntury123",
    dsn="racorap33-scan.corp.intranet:1521/ccdw01p_users",
    encoding="UTF-8")

In [ ]:
%%time
#CBSA WCs

# LU_QUAL_O qualifications for XGS - update date here
query3 = """
            WITH QD AS (
            SELECT  
                    O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.LUID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    ,O.UNI_ECD
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND uni_ecd < '1-DEC-2022'
                    
            )
            
            SELECT  PRIMARY_WIRE_CENTER_ID
                    ,count(distinct LUID) as lu_qual_cnt
                    
            FROM    QD
            
            GROUP BY
                    PRIMARY_WIRE_CENTER_ID

                                """

luqualo_ora_xgs_df = pd.read_sql(query3, con=conn2)

In [ ]:
# luqualo_ora_xgs_df

In [ ]:
%%time

#getting sales data separate because it takes a while to run - update date here

query4 = """

WITH HH AS (         --comes from corp strategy

    select count(t.ntwk_source_key) as lu_cnt
            ,t.snapshot_date
            ,t.WIRE_CENTER_ID
            ,t.CBSA_NAME
            ,t.STATE_CD
            ,CASE WHEN t.qualtech NOT IN ('XGSPON', 'GPON') THEN 'COPPER'
                  ELSE t.qualtech END AS qualtech  --In Tableau dashboard fiber = XGSPON,GPON/ everything else = copper 

    from 
            rpcarte.dd_t5846_hh_fp_sbg t
 
    where 
            t.snapshot_date = '30-NOV-2022'            and t.SBG_MKT_LVL1_CUR = 'MASSMKT'
            and t.LU_COUNTABLE = 'Y'
            and t.HSI_MAX_QUAL <> 0 -- speed filter excluding 0 speed group
            and ILEC_OPS = 'LUMEN'
 
    group by 
            t.snapshot_date
            ,t.WIRE_CENTER_ID
            ,t.CBSA_NAME
            ,t.STATE_CD
            ,CASE WHEN t.qualtech NOT IN ('XGSPON', 'GPON') THEN 'COPPER'
                  ELSE t.qualtech END


), SALES AS (    --Ins: completed sale orders for hsi to xgspon enabled sddresses

    select  sum(t.units) as units
            ,t.transaction_date
            ,t.metric
            ,t.WIRE_CENTER_ID
            ,t.CBSA_NAME
            ,t.STATE_CD
            ,CASE WHEN t.BILLED_TECHNOLOGY = 'Copper' THEN 'COPPER'
                  ELSE t.qualtech END AS qualtech
       
    from    rpcarte.dd_t5846_units_sbg t
    where   t.transaction_date = '30-NOV-2022'
            and t.BILLED_TECHNOLOGY in ('Fiber', 'Copper') -- what technology the customer is actually being billed for
            and t.crnt_gl_cust_type_cd in ('R','S')
            and t.META_SRC_SYS_CD not in ('CTL_ENS_OVRLY') 
            and t.META_SRC_SYS_CD||metric not in ('Q_IR_CPROD_OVRLYEIS')
            and nvl(UPPER(t.PREPAID),'NA')||t.META_SRC_SYS_CD NOT IN ('INSTALINKPPP','INSTALINKQ_IR_CPROD','INSTALINKCTL_ENS')
            and t.ILEC_OPS = 'LUMEN'
            

        
    group by    t.transaction_date
                ,t.metric
                ,t.WIRE_CENTER_ID
                ,t.CBSA_NAME
                ,t.STATE_CD
                ,CASE WHEN t.BILLED_TECHNOLOGY = 'Copper' THEN 'COPPER'
                      ELSE t.qualtech END
                
), EIS AS (

    select  distinct 
            units
            ,transaction_date
            ,WIRE_CENTER_ID
            ,CBSA_NAME
            ,STATE_CD
            ,qualtech
       
    from    SALES
    where   metric = 'EIS'

)

SELECT  distinct
        to_char(h.snapshot_date, 'Mon-YY') as month
        ,h.WIRE_CENTER_ID
        ,h.CBSA_NAME
        ,h.STATE_CD
        ,h.qualtech
        ,h.lu_cnt as HH_cnt
        ,coalesce(e.units,0) as EIS
        ,coalesce((e.units / h.lu_cnt),0) as penetration_rate
        ,coalesce(s.units,0) as Ins_Sales

FROM    HH h
        LEFT JOIN EIS e
            on h.snapshot_date = e.transaction_date
            and h.WIRE_CENTER_ID = e.WIRE_CENTER_ID
            and h.CBSA_NAME = e.CBSA_NAME
            and h.STATE_CD = e.STATE_CD
            and h.qualtech = e.qualtech
        LEFT JOIN SALES s
            on h.snapshot_date = s.transaction_date
            and h.WIRE_CENTER_ID = s.WIRE_CENTER_ID
            and h.CBSA_NAME = s.CBSA_NAME
            and h.STATE_CD = s.STATE_CD
            and h.qualtech = s.qualtech
            and s.metric = 'Ins'

"""



# Best to run 1-3 months of data at a time

sales_ora_df = pd.read_sql(query4, con=conn3)

In [ ]:
# check sales data
#sales_ora_df.to_excel("sales_sep2022_QA.xlsx")
sales_ora_df.head()

In [ ]:
# Import wire center datasource from 'Get Accurate Wire Center Base' project
Sales_WC_excel = pd.read_excel(r'WireCenterBase/FINAL_wire_center_mapping_DO-NOT-ALTER.xlsx',sheet_name = 'Sheet1')

In [ ]:
# clean base table
# check all states are legit
# Sales_WC_excel['STATE'].value_counts()

wc_df = Sales_WC_excel.copy() #save base table

In [ ]:
wc_df = wc_df.rename(columns = {'WIRE_CENTER_ID': 'PRIMARY_WIRE_CENTER_ID',
                                'CBSA_NAME': 'CBSA',
                                'DMA_L2': 'DMA'})

In [ ]:
# Import excel MG readiness data
MG_excel_df = pd.read_excel(r'120122_OpportunityScore_MultiGig_Readiness.xlsx',sheet_name = 'Datasheet')

# Import excel opportunity score data
OppScore_excel_df = pd.read_excel(r'OpportunityScoresMapping/CLLI_ZIP_XREF_WEIGHTED_SCORE.xlsx',sheet_name = 'Final')

In [ ]:
# join in opportunity score
wc_ora_df = wc_df.merge(OppScore_excel_df[['CLLI','Weighted Opportunity Score']], how = 'left', left_on = 'PRIMARY_WIRE_CENTER_ID', right_on = 'CLLI')

In [ ]:
wc_ora_df = wc_ora_df.rename(columns = {'DMA_NAME': 'DMA'})

In [ ]:
# clean data
wc_ora_df = wc_ora_df[['STATE','DMA','CBSA','Weighted Opportunity Score','PRIMARY_WIRE_CENTER_ID']].rename(
    columns = {
        'Weighted Opportunity Score':'OPPORTUNITY_SCORE'})

In [ ]:
# clean mg readiness data - getting core status and monthly planned build - month part needs to be hardcoded manually
MG_excel_df = MG_excel_df[['PRIMARY_WIRE_CENTER_ID','Core Status']].rename(
    columns = {
        'Core Status': 'CORE_READINESS',
})

In [ ]:
# join in planned xgs build/core status

wc_ora_df = wc_ora_df.merge(MG_excel_df, how = 'left', on = 'PRIMARY_WIRE_CENTER_ID')

In [ ]:
# check data
wc_ora_df.head()

In [ ]:
#Import Funding data
Construction_Funding_df = pd.read_excel(r'FundingDetailsExport/Details_Export_Table.xlsx',sheet_name = 'Export')

In [ ]:
Construction_Funding_df['PHASE_ECD_LOGIC'] = Construction_Funding_df['PHASE_ECD_LOGIC'].dt.to_period('M')
Construction_Funding_df['PHASE_ECD_LOGIC'] = Construction_Funding_df['PHASE_ECD_LOGIC'].astype(str)
Construction_pivot = Construction_Funding_df.groupby(by = ["PRIMARY_WIRE_CENTER_ID","PHASE_ECD_LOGIC"], as_index = False).sum()
Construction_pivot = Construction_pivot.pivot(index='PRIMARY_WIRE_CENTER_ID', columns='PHASE_ECD_LOGIC')['PHASE_NUM_OF_LUS'].reset_index()
Construction_pivot = Construction_pivot.fillna(0)

In [ ]:
# check this before manual date update below
Construction_pivot.sum()

In [ ]:
# rename columns
Construction_Funding_df = Construction_pivot.copy()
Construction_Funding_df = Construction_Funding_df.groupby(
    by = ["PRIMARY_WIRE_CENTER_ID"], as_index = False).sum().rename(
    columns = {
        '2022-09': 'SEP-22_XGS_CONSTRUCTION_FUNDED',
        '2022-10': 'OCT-22_XGS_CONSTRUCTION_FUNDED',
        '2022-11': 'NOV-22_XGS_CONSTRUCTION_FUNDED',
        '2022-12': 'DEC-22_XGS_CONSTRUCTION_FUNDED',
        '2023-01': 'JAN-23_XGS_CONSTRUCTION_FUNDED',
        '2023-02': 'FEB-23_XGS_CONSTRUCTION_FUNDED',
        '2023-03': 'MAR-23_XGS_CONSTRUCTION_FUNDED',
        '2023-04': 'APR-23_XGS_CONSTRUCTION_FUNDED',
        '2023-05': 'MAY-23_XGS_CONSTRUCTION_FUNDED',
        '2023-06': 'JUN-23_XGS_CONSTRUCTION_FUNDED',
        '2023-07': 'JUL-23_XGS_CONSTRUCTION_FUNDED',
        '2023-08': 'AUG-23_XGS_CONSTRUCTION_FUNDED',
        '2023-09': 'SEP-23_XGS_CONSTRUCTION_FUNDED',
        '2023-10': 'OCT-23_XGS_CONSTRUCTION_FUNDED',
        '2023-11': 'NOV-23_XGS_CONSTRUCTION_FUNDED',
        '2023-12': 'DEC-23_XGS_CONSTRUCTION_FUNDED',
})

In [ ]:
# join in construction funding to base table

wc_ora_df = wc_ora_df.merge(Construction_Funding_df, how = 'left', on = 'PRIMARY_WIRE_CENTER_ID')

## XGS enablement data

In [ ]:
# join in actual build for XGS - manually update date column name

wc_ora_df = wc_ora_df.merge(luqualo_ora_xgs_df[['PRIMARY_WIRE_CENTER_ID','LU_QUAL_CNT']], 
                            how = 'left', 
                            on = 'PRIMARY_WIRE_CENTER_ID').rename(
    columns = {
        'LU_QUAL_CNT': 'NOV-22_ENABLED_XGS_LU'})

# clean data/reorder columns - manually adjust date column names
wc_ora_df = wc_ora_df.rename(
    columns = {
        'Weighted Opportunity Score':'OPPORTUNITY_SCORE'})

## Salesforce XGS data

In [ ]:
# Import Funding data
salesforce_xgs_df = pd.read_excel(r'SalesforceExport/salesdata.xlsx',sheet_name = 'Ella XGS Sales')
active_xgs_subscribers_df = salesforce_xgs_df[(salesforce_xgs_df['Completion Date'].isna()==False)&(salesforce_xgs_df['Account Status']=='Active')]
active_xgs_subscribers_df['Completion Date'] = pd.to_datetime(active_xgs_subscribers_df['Completion Date']).dt.to_period('M')

In [ ]:
# manually update month mask
month_mask = active_xgs_subscribers_df['Completion Date'] < '2022-12'

active_xgs_subscribers_month_df = active_xgs_subscribers_df[month_mask].groupby(
    ['Wire Center'], 
    as_index=False)['Work Order Number'].nunique()

In [ ]:
# manually update month field name
active_xgs_subscribers_month_df.rename(
    columns = {
        'Wire Center':'PRIMARY_WIRE_CENTER_ID',
        'Work Order Number':'NOV-22_XGS_ACTIVE_SUBS'}, inplace=True)

In [ ]:
wc_ora_df = wc_ora_df.merge(active_xgs_subscribers_month_df,
                how='left',
                on='PRIMARY_WIRE_CENTER_ID')

In [ ]:
# join in subscribers data sales_ora_df
sales_copper = sales_ora_df.loc[sales_ora_df['QUALTECH']=='COPPER',['WIRE_CENTER_ID','HH_CNT','EIS']].groupby(
    by = 'WIRE_CENTER_ID', as_index = False).sum()

In [ ]:
sales_gpon = sales_ora_df.loc[sales_ora_df['QUALTECH']=='GPON',['WIRE_CENTER_ID','HH_CNT','EIS']].groupby(
    by = 'WIRE_CENTER_ID', as_index = False).sum()
sales_xgs = sales_ora_df.loc[sales_ora_df['QUALTECH']=='XGSPON',['WIRE_CENTER_ID','HH_CNT','EIS']].groupby(
    by = 'WIRE_CENTER_ID', as_index = False).sum()

In [ ]:
# join in gpon sales - update date name
wc_ora_df = wc_ora_df.merge(sales_gpon, how = 'left', 
                            left_on = 'PRIMARY_WIRE_CENTER_ID', 
                            right_on = 'WIRE_CENTER_ID').drop('WIRE_CENTER_ID', axis = 1).rename(
    columns = {
        'HH_CNT':'NOV-22_GPON_ENABLED_HH',
        'EIS':'NOV-22_GPON_ACTIVE_SUBS'})

In [ ]:
# join in copper sales - update date name
wc_ora_df = wc_ora_df.merge(sales_copper, 
                            how = 'left', 
                            left_on = 'PRIMARY_WIRE_CENTER_ID', 
                            right_on = 'WIRE_CENTER_ID').drop('WIRE_CENTER_ID', axis = 1).rename(
    columns = {
        'HH_CNT':'NOV-22_COPPER_ENABLED_HH',
        'EIS':'NOV-22_COPPER_ACTIVE_SUBS'})

In [ ]:
wc_ora_df[['OPPORTUNITY_SCORE','CORE_READINESS']] = wc_ora_df[['OPPORTUNITY_SCORE','CORE_READINESS']].fillna('N/A')

In [ ]:
# missing xgs active subs, planned xgs for entire year
wc_ora_df.columns

In [ ]:
import time

# export
TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate +"_WC_Universe_Cumulative_Rpt_Data_DEC22.xlsx"

with pd.ExcelWriter(excelfilename) as writer:  
    wc_ora_df.to_excel(writer, sheet_name='Cumulative Data', index = False)
    luqualo_ora_xgs_df.to_excel(writer, sheet_name='LUQUAL Build Details XGS', index = False)
    Construction_Funding_df.to_excel(writer, sheet_name='ST Construction Funding Details', index = False)
    sales_ora_df.to_excel(writer, sheet_name='Sales Data Details', index = False)
    salesforce_xgs_df.to_excel(writer, sheet_name='Salesforce Details', index = False)